In [ ]:
from qiskit import Aer, QuantumCircuit
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit_machine_learning.neural_networks import CircuitQNN
from qiskit_machine_learning.connectors import TorchConnector
from torch.optim import Adam
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn import metrics

# Define seed for reproducibility
algorithm_globals.random_seed = 42

# Load and prepare the data
data_path = "path/to/your/sample.csv"  # Update this path to your dataset location
df = pd.read_csv(data_path)

# Assuming 'y' is the label column
df_labels = df['y']
df_features = df.drop(['y'], axis=1)

# Split dataset into train and test
sample_train, sample_test, label_train, label_test = train_test_split(df_features, df_labels, test_size=0.2, random_state=22)

# Apply PCA to reduce dimensions
n_dim = 2
pca = PCA(n_components=n_dim).fit(sample_train)
sample_train = pca.transform(sample_train)
sample_test = pca.transform(sample_test)

# Normalize the data
std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)

# Scale for better fit within the feature map
samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)

# Define the feature map
feature_map = RawFeatureVector(feature_dimension=n_dim)

# Define the variational circuit
var_circuit = TwoLocal(n_dim, 'ry', 'cz', reps=3, entanglement='full')

# Define the quantum instance
backend = QuantumInstance(Aer.get_backend('aer_simulator_statevector'))

# Define the QNN
qnn = CircuitQNN(circuit=var_circuit, input_params=feature_map.parameters, weight_params=var_circuit.parameters, quantum_instance=backend)

# Convert QNN to PyTorch model
torch_qnn = TorchConnector(qnn)
model = torch.nn.Sequential(
    torch_qnn,
    torch.nn.Linear(qnn.output_shape[0], 2)
)

# Define loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

# Convert data to PyTorch tensors
X_train = torch.tensor(sample_train, dtype=torch.float32)
y_train = torch.tensor(label_train.values, dtype=torch.long)
X_test = torch.tensor(sample_test, dtype=torch.float32)
y_test = torch.tensor(label_test.values, dtype=torch.long)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = loss_fn(output, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Evaluation
model.eval()
with torch.no_grad():
    test_output = model(X_test)
    predictions = torch.argmax(test_output, dim=1)
    confidence_scores = torch.nn.functional.softmax(test_output, dim=1).max(dim=1)[0].numpy()
    predicted_labels = predictions.numpy()

# Generate output table
output_table = pd.DataFrame({
    "Sample": range(1, len(predicted_labels) + 1),
    "Predicted Label": ["Positive" if label == 1 else "Negative" for label in predicted_labels],
    "Confidence Score": confidence_scores
})

print(output_table)
